In [1]:
import pandas as pd

In [2]:
man_df = pd.read_csv('data/caesar_fr.csv', encoding='utf-8', sep=';')
man_df=man_df.sample(frac=1)
caesar_male = man_df[man_df['sex'] == 1]
caesar_male = caesar_male.drop(columns=["sex"], errors='ignore')

In [3]:
caesar_male=caesar_male.loc[:, (caesar_male != 0).any()]

In [4]:
import pandas as pd
import numpy as np

def remplacer_nan_par_moyenne(df):
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            moyenne = df[col].mean(skipna=True)
            df[col] = df[col].fillna(moyenne)
    return df

caesar_malecaesar_malecaesar_male = remplacer_nan_par_moyenne(caesar_male)
caesar_male.head()

,taille,age,weight,tour_de_cheville,longueur_du_bras,tour_de_poitrine,tour_de_poitrine_mesure_a_la_main,hauteur_de_poitrine,hauteur_d_entrejambe,hauteur_des_hanches,hauteur_des_genoux,tour_du_cou,largeur_d_epaule,hauteur_des_epaules,largeur_des_epaules_a_l_horizontales,longueur_de_la_colonne_vertebrale_jusqu_au_poignet,tour_de_cuisse,tour_de_taille,hauteur_de_la_taille,longueur_d_avant_bras
498,190.5000,44.0,95.481116,26.3906,69.6976,108.4072,NaN,134.7978,84.9122,92.5068,52.2986,46.2026,42.9768,156.5910,50.4952,62.1411,60.4012,98.2980,109.0930,26.8605
31,180.0098,33.0,87.543256,27.5082,66.3956,105.4100,NaN,132.7912,83.9978,93.3958,50.6984,43.1038,42.7228,148.0058,53.1876,59.7154,61.7982,92.3036,110.5916,27.1018
2180,167.7924,40.0,80.739376,27.6098,62.4078,104.3940,NaN,121.0056,72.6948,86.2076,47.9044,46.8884,40.1828,138.0998,48.2092,57.3278,57.9882,90.8050,98.6028,26.6827
206,189.9920,21.0,85.098395,26.1874,69.9008,101.0920,NaN,138.6078,88.5952,91.8972,51.4477,50.1904,43.7134,156.9847,49.3014,63.3857,60.9092,89.6112,114.3000,26.2509
1240,177.0126,34.0,103.418976,26.1874,64.9986,113.3094,NaN,129.5908,78.9940,85.4964,49.3014,45.6946,39.0906,145.7071,48.3108,60.0964,73.1012,105.6894,100.5078,27.1272


In [5]:
caesar_male["tour_de_cuisse_ratio"] = caesar_male["tour_de_cuisse"] / caesar_male["taille"]
caesar_male["tour_de_taille_ratio"] = caesar_male["tour_de_taille"] / caesar_male["taille"]
caesar_male["largeur_d_epaule_ratio"] = caesar_male["largeur_d_epaule"] / caesar_male["taille"]

In [6]:
caesar_male["tour_de_cuisse_ratio"].describe() 


count    2107.000000
mean        0.333812
std         0.031655
min         0.248715
25%         0.312557
50%         0.331693
75%         0.350709
max         0.536801
Name: tour_de_cuisse_ratio, dtype: float64

In [7]:
def classifier_morphologie(row):
    # Extraction des mesures
    #tour_ventre = row['tour_de_ventre_maximal']
    tour_taille = row['tour_de_taille_ratio']
    forme_poitrine = row['largeur_d_epaule_ratio']
    tour_cuisse = row['tour_de_cuisse_ratio']
    
    
    # --- Catégorie VENTRE (basée sur le ratio ventre/taille) ---
    if tour_taille < 0.462864:
        ventre = "plat"
    elif 0.462864 <= tour_taille<0.536128:
        ventre = "moyen"
    else:
        ventre = "rond"
    
    # --- Catégorie TORSE (basée sur le tour de poitrine seul ou comparé à la taille) ---
    if forme_poitrine < 0.224841:
        torse = "fin"
    elif 0.224841 <=forme_poitrine < 0.242234 :
        torse = "moyen"
    else:
        torse = "large"
    
    # --- Catégorie CUISSES (ratio cuisse / hanches) ---
    if tour_cuisse < 0.312557:
        cuisses = "fines"
    elif  0.312557 <=tour_cuisse<0.350709 :
        cuisses = "moyennes"
    else:
        cuisses = "larges"
    
    return pd.Series([ventre, torse, cuisses], 
                     index=['categorie_ventre', 'categorie_torse', 'categorie_cuisses'])
    
# Remplace `df` par le nom de ton DataFrame
caesar_male[['categorie_ventre', 'categorie_torse', 'categorie_cuisses']] = caesar_male.apply(classifier_morphologie, axis=1)

In [8]:
from sklearn.preprocessing import RobustScaler

# Colonnes à exclure
cols_to_exclude = ["taille","age","weight","categorie_ventre", "categorie_torse", "categorie_cuisses"]

# Colonnes à scaler (tout sauf celles à exclure)

caesar_male = caesar_male.dropna(how='all', axis=1)
cols_to_scale = [col for col in caesar_male.columns if col not in cols_to_exclude]
# On copie le DataFrame pour garder l'original intact
df_scaled = caesar_male.copy()

# On applique RobustScaler uniquement sur les colonnes numériques à scaler
scaler = RobustScaler()
df_scaled[cols_to_scale] = scaler.fit_transform(df_scaled[cols_to_scale])

# Résultat : df_scaled contient les colonnes scalées (sauf celles exclues)


In [9]:
# Supprime toutes les colonnes contenant "ratio" dans leur nom
colonnes_ratio = [col for col in df_scaled.columns if "ratio" in col]
df_scaled.drop(columns=colonnes_ratio, inplace=True)

print(f"✅ Colonnes supprimées : {colonnes_ratio}")

✅ Colonnes supprimées : ['tour_de_cuisse_ratio', 'tour_de_taille_ratio', 'largeur_d_epaule_ratio']


In [10]:
df_scaled.head()

,taille,age,weight,tour_de_cheville,longueur_du_bras,tour_de_poitrine,hauteur_de_poitrine,hauteur_d_entrejambe,hauteur_des_hanches,hauteur_des_genoux,...,hauteur_des_epaules,largeur_des_epaules_a_l_horizontales,longueur_de_la_colonne_vertebrale_jusqu_au_poignet,tour_de_cuisse,tour_de_taille,hauteur_de_la_taille,longueur_d_avant_bras,categorie_ventre,categorie_torse,categorie_cuisses
498,190.5000,44.0,95.481116,-0.101266,1.237569,0.604607,0.703650,0.537415,0.591973,0.699523,...,1.132933,0.569832,0.912442,0.242754,0.733083,0.654723,0.240741,moyen,moyen,moyennes
31,180.0098,33.0,87.543256,0.455696,0.519337,0.378119,0.472993,0.414966,0.709030,0.298887,...,0.229793,1.162011,0.325653,0.442029,0.289474,0.846906,0.358025,moyen,moyen,moyennes
2180,167.7924,40.0,80.739376,0.506329,-0.348066,0.301344,-0.881752,-1.098639,-0.237458,-0.400636,...,-0.812291,0.067039,-0.251920,-0.101449,0.178571,-0.690554,0.154321,rond,moyen,moyennes
206,189.9920,21.0,85.098395,-0.202532,1.281768,0.051823,1.141606,1.030612,0.511706,0.486486,...,1.174349,0.307263,1.213518,0.315217,0.090226,1.322476,-0.055556,moyen,moyen,moyennes
1240,177.0126,34.0,103.418976,-0.202532,0.215470,0.975048,0.105109,-0.255102,-0.331104,-0.050874,...,-0.012024,0.089385,0.417819,2.054348,1.280075,-0.446254,0.370370,rond,fin,larges


In [11]:
df_scaled.columns

Index(['taille', 'age', 'weight', 'tour_de_cheville', 'longueur_du_bras',
       'tour_de_poitrine', 'hauteur_de_poitrine', 'hauteur_d_entrejambe',
       'hauteur_des_hanches', 'hauteur_des_genoux', 'tour_du_cou',
       'largeur_d_epaule', 'hauteur_des_epaules',
       'largeur_des_epaules_a_l_horizontales',
       'longueur_de_la_colonne_vertebrale_jusqu_au_poignet', 'tour_de_cuisse',
       'tour_de_taille', 'hauteur_de_la_taille', 'longueur_d_avant_bras',
       'categorie_ventre', 'categorie_torse', 'categorie_cuisses'],
      dtype='object')

In [12]:
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib


X_train=df_scaled[['taille', 'age', 'weight',
              'categorie_ventre', 'categorie_torse', 'categorie_cuisses']]

# Colonnes numériques et catégorielles
cols_num = ["taille","age","weight"]
cols_cat = ["categorie_ventre", "categorie_torse", "categorie_cuisses"]

# Pipeline pour les colonnes numériques : imputation par médiane + scaling
pipeline_num = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

# Pipeline pour les colonnes catégorielles : imputation par la valeur la plus fréquente + one-hot
pipeline_cat = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Pipeline globale de prétraitement
preprocessor = ColumnTransformer(transformers=[
    ("num", pipeline_num, cols_num),
    ("cat", pipeline_cat, cols_cat)
])

# Fit sur X_train (tu dois définir X_train comme DataFrame brut)
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['taille', 'age', 'weight']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['categorie_ventre', 'categorie_torse',
                                  'categorie_cuisses'])])

In [13]:
print("Colonnes dans X_train :", X_train.columns.tolist())
print("Colonnes numériques :", cols_num)
print("Colonnes catégorielles :", cols_cat)

Colonnes dans X_train : ['taille', 'age', 'weight', 'categorie_ventre', 'categorie_torse', 'categorie_cuisses']
Colonnes numériques : ['taille', 'age', 'weight']
Colonnes catégorielles : ['categorie_ventre', 'categorie_torse', 'categorie_cuisses']


In [14]:
import joblib
import os

# 📁 Répertoire contenant les fichiers .pkl
dossier_modeles = "./modeles_tpot_complet"  # modifie ce chemin si nécessaire

# 📌 Chargement de tous les modèles
modeles = {}
for nom_fichier in os.listdir(dossier_modeles):
    if nom_fichier.endswith(".pkl"):
        nom_cible = nom_fichier.replace("tpot_", "").replace(".pkl", "")
        chemin_complet = os.path.join(dossier_modeles, nom_fichier)
        modeles[nom_cible] = joblib.load(chemin_complet)

print(f"✅ {len(modeles)} modèles chargés :", list(modeles.keys()))

✅ 16 modèles chargés : ['hauteur_des_epaules', 'hauteur_des_genoux', 'hauteur_des_hanches', 'hauteur_de_la_taille', 'hauteur_de_poitrine', 'hauteur_d_entrejambe', 'largeur_des_epaules_a_l_horizontales', 'largeur_d_epaule', 'longueur_de_la_colonne_vertebrale_jusqu_au_poignet', 'longueur_du_bras', 'longueur_d_avant_bras', 'tour_de_cheville', 'tour_de_cuisse', 'tour_de_poitrine', 'tour_de_taille', 'tour_du_cou']


In [15]:
import os

dossier_modeles = "./modeles_tpot_complet"  # adapte ce chemin
noms_fichiers = [f for f in os.listdir(dossier_modeles) if f.endswith(".pkl")]

# Dossier de sauvegarde des pipelines complets
dossier_output = "pipelines_all_dataset"
os.makedirs(dossier_output, exist_ok=True)

for nom_fichier in noms_fichiers:
    chemin_modele = os.path.join(dossier_modeles, nom_fichier)
    
    # Charger le modèle TPOT
    tpot_model = joblib.load(chemin_modele)
    
    # Créer le pipeline complet
    full_pipeline = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", tpot_model)
    ])
    
    # Sauvegarder le pipeline complet
    nom_fichier_pipeline = f"pipeline_{nom_fichier}"
    chemin_output = os.path.join(dossier_output, nom_fichier_pipeline)
    joblib.dump(full_pipeline, chemin_output)

    print(f"✔ Pipeline sauvegardé : {chemin_output}")

✔ Pipeline sauvegardé : pipelines_all_dataset\pipeline_tpot_hauteur_des_epaules.pkl
✔ Pipeline sauvegardé : pipelines_all_dataset\pipeline_tpot_hauteur_des_genoux.pkl
✔ Pipeline sauvegardé : pipelines_all_dataset\pipeline_tpot_hauteur_des_hanches.pkl
✔ Pipeline sauvegardé : pipelines_all_dataset\pipeline_tpot_hauteur_de_la_taille.pkl
✔ Pipeline sauvegardé : pipelines_all_dataset\pipeline_tpot_hauteur_de_poitrine.pkl
✔ Pipeline sauvegardé : pipelines_all_dataset\pipeline_tpot_hauteur_d_entrejambe.pkl
✔ Pipeline sauvegardé : pipelines_all_dataset\pipeline_tpot_largeur_des_epaules_a_l_horizontales.pkl
✔ Pipeline sauvegardé : pipelines_all_dataset\pipeline_tpot_largeur_d_epaule.pkl
✔ Pipeline sauvegardé : pipelines_all_dataset\pipeline_tpot_longueur_de_la_colonne_vertebrale_jusqu_au_poignet.pkl
✔ Pipeline sauvegardé : pipelines_all_dataset\pipeline_tpot_longueur_du_bras.pkl
✔ Pipeline sauvegardé : pipelines_all_dataset\pipeline_tpot_longueur_d_avant_bras.pkl
✔ Pipeline sauvegardé : pipelin

In [16]:
import os
import joblib
import pandas as pd

# 📁 1. Dossier où se trouvent les 22 pipelines
dossier_pipelines = "pipelines_all_dataset"  # adapte selon ton cas

# 🧠 2. Charger tous les pipelines .pkl
pipelines = {}
for filename in os.listdir(dossier_pipelines):
    if filename.endswith(".pkl"):
        path = os.path.join(dossier_pipelines, filename)
        nom_variable = filename.replace("pipeline_tpot_", "").replace(".pkl", "")
        pipelines[nom_variable] = joblib.load(path)

# 📦 3. Exemple de nouvelles données
new_data = pd.DataFrame([{
    "taille": 187,
    "weight": 80,
    "age": 33,
    "categorie_ventre": "moyen",
    "categorie_cuisses": "medium",
    "categorie_torse": "large"
}])

# 🧮 4. Appliquer chaque pipeline à la même ligne de données
predictions = {}
for nom_variable, pipeline in pipelines.items():
    y_pred = pipeline.predict(new_data)
    predictions[nom_variable] = y_pred[0]  # extraire la valeur unique

# 🖨️ 5. Résultat
print("Prédictions :")
for var, val in predictions.items():
    print(f"{var} = {val:.2f}")


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MaxAbsScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-datase

Prédictions :
hauteur_des_epaules = 152.40
hauteur_des_genoux = 52.46
hauteur_des_hanches = 93.51
hauteur_de_la_taille = 111.03
hauteur_de_poitrine = 135.35
hauteur_d_entrejambe = 87.33
largeur_des_epaules_a_l_horizontales = 48.46
largeur_d_epaule = 43.23
longueur_de_la_colonne_vertebrale_jusqu_au_poignet = 60.74
longueur_du_bras = 66.66
longueur_d_avant_bras = 27.62
tour_de_cheville = 27.13
tour_de_cuisse = 59.55
tour_de_poitrine = 96.51
tour_de_taille = 83.59
tour_du_cou = 46.71


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MaxAbsScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
